<h1>Combine csv's<h1>

#### If you have multiple csv's

In [ ]:
import os
import pandas as pd

dirpath = "input path to the folder of your csv's"

df = pd.concat([pd.read_csv(os.path.join(dirpath, fname),compression ="gzip")
                for fname in os.listdir(dirpath)], ignore_index=True)

df.to_csv("output path for the csv.gzip")

<h2>Change timestamp to millisecond<h2>

In [ ]:
from datetime import datetime

startTime = 1648806250315

def formatTimestamp(timestamp):

    date_format = "%Y-%m-%d %H:%M:%S.%f"
    try:
        timestamp = datetime.strptime(timestamp[:-4], date_format).timestamp() #timezone removal

    except ValueError:
        timestamp = datetime.strptime(timestamp[:-4], date_format[:-3]).timestamp() #time zone removal if no timestamp decimal

    timestamp *= 1000.0
    timestamp = int(timestamp)

    timestamp -= startTime
    return timestamp

<h2>Convert Colors to Key-Value Pair<h2>

In [ ]:
def formatPixelColor(pixel_color):
    hex_to_key = {
        "#000000": 0,
        "#00756F": 1,
        "#009EAA": 2,
        "#00A368": 3,
        "#00CC78": 4,
        "#00CCC0": 5,
        "#2450A4": 6,
        "#3690EA": 7,
        "#493AC1": 8,
        "#515252": 9,
        "#51E9F4": 10,
        "#6A5CFF": 11,
        "#6D001A": 12,
        "#6D482F": 13,
        "#7EED56": 14,
        "#811E9F": 15,
        "#898D90": 16,
        "#94B3FF": 17,
        "#9C6926": 18,
        "#B44AC0": 19,
        "#BE0039": 20,
        "#D4D7D9": 21,
        "#DE107F": 22,
        "#E4ABFF": 23,
        "#FF3881": 24,
        "#FF4500": 25,
        "#FF99AA": 26,
        "#FFA800": 27,
        "#FFB470": 28,
        "#FFD635": 29,
        "#FFF8B8": 30,
        "#FFFFFF": 31,
    }

    return hex_to_key[pixel_color]

<h2>Split Pixel Coordiantes into X and Y Columns<h2>

In [ ]:
def splitCoords(value):

    value["coordinate"] = value["coordinate"].apply(lambda x: x.split(","))

    value["x"] = value["coordinate"].apply(lambda x: x[0]).astype("uint16")
    value["y"] = value["coordinate"].apply(lambda x: x[1]).astype("uint16")

    del value["coordinate"]
    
    return value

<h3>The mods were able to place rectangles which consist of 2 x coords and 2 y coords. We need to split the values so that all pixels inside are drawn<h3>

<h2>Split Rectangle Coordinates into X and Y Columns<h2>

In [ ]:
# rectangles formatted into x1, x2, y1, y2 tuple
# we turn into in x,y value of every pixel placed

def SplitCoordsRectangles(rectangles):
    rectangles["coordinate"] = rectangles["coordinate"].apply(
        lambda x: [int(c) for c in x.split(",")]
    )

    recsValue = pd.DataFrame(columns=["timestamp", "pixel_color", "x", "y"])

    # Iterate over the rectangles in the chunk.
    for rect in rectangles.itertuples():
        x1, y1, x2, y2 = rect.coordinate
        width = x2 - x1 + 1 # plus 1 since we start at 0
        height = y2 - y1 + 1

        for i in range(width):
            for j in range(height):
                x = x1 + i
                y = y1 + j

                recsValue.loc[len(recsValue)] = [
                    rect.timestamp,
                    rect.pixel_color,
                    x,
                    y,
                ]

    recsValue["timestamp"] = recsValue["timestamp"].astype("uint32")
    recsValue["pixel_color"] = recsValue["pixel_color"].astype("uint8")
    recsValue["x"] = recsValue["x"].astype("uint16")
    recsValue["y"] = recsValue["y"].astype("uint16")

    return recsValue

<h2>Process The Data Chunks<h2>

In [ ]:
def processChunk(chunk, df):
    
    chunk["timestamp"] = chunk["timestamp"].astype("uint32")
    chunk["pixel_color"] = chunk["pixel_color"].astype("uint8")

    
    groups = chunk.groupby(chunk["coordinate"].str.count(",") == 1)
    rectangles = None
    points = groups.get_group(True).reset_index(drop=True)
    try:
        rectangles = groups.get_group(False).reset_index(drop=True)
    except KeyError:
        
        pass

    points = splitCoords(points)

    df = pd.concat((df, points), ignore_index=True)

    if rectangles is not None:
        rectangles = SplitCoordsRectangles(rectangles)
        df = pd.concat((df, rectangles), ignore_index=True)

    return df


chunk = pd.DataFrame(
    {
        "timestamp": [123, 456, 789],
        "coordinate": ["0,0", "1,1", "2,2,3,3"],
        "pixel_color": [0, 1, 2],
    }
)

<h1>Runner<h1>
<h4>this may take 1hr plus...<h4>

In [ ]:
chunkSize = 10000000

def runner(file, outFilePath):
    df = pd.DataFrame(columns=["timestamp", "pixel_color", "x","y"])
    df["timestamp"] = df["timestamp"].astype("uint32")
    df["pixel_color"] = df["pixel_color"].astype("uint8")
    df["x"] = df["x"].astype("uint16")
    df["y"] = df["y"].astype("uint16")

    with pd.read_csv(file, usecols=["timestamp","pixel_color","coordinate"], converters= {"timestamp": formatTimestamp, "pixel_color": formatPixelColor}, 
    chunksize=chunkSize, 
    compression="gzip", 
    engine="c"
    ) as csv: #compression="gzip"

        for chunk in csv:
            df = processChunk(chunk, df)

    df.sort_values("timestamp", inplace=True, ignore_index=True, version="2.6")

    df.to_parquet(outFilePath)

    return df

outFilePath = "path to output parquet file"
file = "path to your csv.gzip file"

runner(file, outFilePath)

df = pd.read_parquet(outFilePath)
df

<h2>Change timestamp to millisecond<h2>

In [5]:
from datetime import datetime

startTime = 1648806250315

def formatTimestamp(timestamp):

    date_format = "%Y-%m-%d %H:%M:%S.%f"
    try:
        timestamp = datetime.strptime(timestamp[:-4], date_format).timestamp() #timezone removal

    except ValueError:
        timestamp = datetime.strptime(timestamp[:-4], date_format[:-3]).timestamp() #time zone removal if no timestamp decimal

    timestamp *= 1000.0
    timestamp = int(timestamp)

    timestamp -= startTime
    return timestamp

<h2>Convert Colors to Key-Value Pair<h2>

In [6]:
def formatPixelColor(pixel_color):
    hex_to_key = {
        "#000000": 0,
        "#00756F": 1,
        "#009EAA": 2,
        "#00A368": 3,
        "#00CC78": 4,
        "#00CCC0": 5,
        "#2450A4": 6,
        "#3690EA": 7,
        "#493AC1": 8,
        "#515252": 9,
        "#51E9F4": 10,
        "#6A5CFF": 11,
        "#6D001A": 12,
        "#6D482F": 13,
        "#7EED56": 14,
        "#811E9F": 15,
        "#898D90": 16,
        "#94B3FF": 17,
        "#9C6926": 18,
        "#B44AC0": 19,
        "#BE0039": 20,
        "#D4D7D9": 21,
        "#DE107F": 22,
        "#E4ABFF": 23,
        "#FF3881": 24,
        "#FF4500": 25,
        "#FF99AA": 26,
        "#FFA800": 27,
        "#FFB470": 28,
        "#FFD635": 29,
        "#FFF8B8": 30,
        "#FFFFFF": 31,
    }

    return hex_to_key[pixel_color]

<h2>Split Coordiantes into X and Y column<h2>

In [7]:
def splitCoords(value):

    value["coordinate"] = value["coordinate"].apply(lambda x: x.split(","))

    value["x"] = value["coordinate"].apply(lambda x: x[0]).astype("uint16")
    value["y"] = value["coordinate"].apply(lambda x: x[1]).astype("uint16")

    del value["coordinate"]
    
    return value

<h2>Pepemods ... They place rectangles, we make new df of rectangles<h2> 

In [8]:
# rectangles formatted into x1, x2, y1, y2 tuple
# we trun into in x,y value of every pixel placed

def SplitCoordsRectangles(rectangles):
    rectangles["coordinate"] = rectangles["coordinate"].apply(
        lambda x: [int(c) for c in x.split(",")]
    )

    recsValue = pd.DataFrame(columns=["timestamp", "pixel_color", "x", "y"])

    # Iterate over the rectangles in the chunk.
    for rect in rectangles.itertuples():
        x1, y1, x2, y2 = rect.coordinate
        width = x2 - x1 + 1 # plus 1 since we start at 0
        height = y2 - y1 + 1

        for i in range(width):
            for j in range(height):
                x = x1 + i
                y = y1 + j

                recsValue.loc[len(recsValue)] = [
                    rect.timestamp,
                    rect.pixel_color,
                    x,
                    y,
                ]

    recsValue["timestamp"] = recsValue["timestamp"].astype("uint32")
    recsValue["pixel_color"] = recsValue["pixel_color"].astype("uint8")
    recsValue["x"] = recsValue["x"].astype("uint16")
    recsValue["y"] = recsValue["y"].astype("uint16")

    return recsValue

In [9]:
def processChunk(chunk, df):
    
    chunk["timestamp"] = chunk["timestamp"].astype("uint32")
    chunk["pixel_color"] = chunk["pixel_color"].astype("uint8")

    
    groups = chunk.groupby(chunk["coordinate"].str.count(",") == 1)
    rectangles = None
    points = groups.get_group(True).reset_index(drop=True)
    try:
        rectangles = groups.get_group(False).reset_index(drop=True)
    except KeyError:
        
        pass

    points = splitCoords(points)

    df = pd.concat((df, points), ignore_index=True)

    if rectangles is not None:
        rectangles = SplitCoordsRectangles(rectangles)
        df = pd.concat((df, rectangles), ignore_index=True)

    return df


chunk = pd.DataFrame(
    {
        "timestamp": [123, 456, 789],
        "coordinate": ["0,0", "1,1", "2,2,3,3"],
        "pixel_color": [0, 1, 2],
    }
)



<h1>Runner<h1>

In [ ]:
#this may take 1hr plus...

In [10]:
chunkSize = 10000000

def runner(file, outFilePath):
    df = pd.DataFrame(columns=["timestamp", "pixel_color", "x","y"])
    df["timestamp"] = df["timestamp"].astype("uint32")
    df["pixel_color"] = df["pixel_color"].astype("uint8")
    df["x"] = df["x"].astype("uint16")
    df["y"] = df["y"].astype("uint16")

    with pd.read_csv(file, usecols=["timestamp","pixel_color","coordinate"], converters= {"timestamp": formatTimestamp, "pixel_color": formatPixelColor}, 
    chunksize=chunkSize, 
    compression="gzip", 
    engine="c"
    ) as csv: #compression="gzip"

        for chunk in csv:
            df = processChunk(chunk, df)

    df.sort_values("timestamp", inplace=True, ignore_index=True, version="2.6")

    df.to_parquet(outFilePath)

    return df

outFilePath = "path to output parquet file"
file = "path to your csv.gzip file"

runner(file, outFilePath)

df = pd.read_parquet(outFilePath)
df